<a href="https://colab.research.google.com/github/Nuwantha97/Sinhala_spell_and_grammer_checker/blob/Notebooks/Context_Awareness_with_spell_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import pipeline

# Load a fill-mask pipeline for multilingual model (supports Sinhala)
fill_mask = pipeline("fill-mask", model="xlm-roberta-base")

# Input sentence with a blank, using the correct mask token
sentence = "මම " + fill_mask.tokenizer.mask_token + " පාසැලට බස් මගින් යනවා."
# or
# sentence = f"මම {fill_mask.tokenizer.mask_token} පාසැලට බස් මගින් යනවා."

# Get predictions
predictions = fill_mask(sentence)

# Display predictions
for pred in predictions:
    print(f"Word: {pred['token_str']}, Confidence: {pred['score']:.4f}")